In [1]:
val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
// note: use sbt assembly to create jar
interp.load.cp(os.Path(path))
import $ivy.`org.plotly-scala::plotly-almond:0.8.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import ujson.Value
import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._



path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                      

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import ujson.Value

import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")

import $ivy.$                                         

import better.files._



In [2]:
import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.RunConfig
import upickle.default.{macroRW, read, write, ReadWriter => RW}
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._
import edu.colorado.plv.bounder.{BounderUtil,TestSpec,PickleSpec}
import edu.colorado.plv.bounder.symbolicexecutor.state.ReceiverNonNull
import edu.colorado.plv.bounder.symbolicexecutor.state.DisallowedCallin
import edu.colorado.plv.bounder.symbolicexecutor.state.InitialQuery
import edu.colorado.plv.bounder.lifestate.LifeState.LSSpec
import edu.colorado.plv.bounder.lifestate.{FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSpace, ViewSpec}

import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.RunConfig

import upickle.default.{macroRW, read, write, ReadWriter => RW}

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

import edu.colorado.plv.bounder.{BounderUtil,TestSpec,PickleSpec}

import edu.colorado.plv.bounder.symbolicexecutor.state.ReceiverNonNull

import edu.colorado.plv.bounder.symbolicexecutor.state.DisallowedCallin

import edu.colorado.plv.bounder.symbolicexecutor.state.InitialQuery

import edu.colorado.plv.bounder.lifestate.LifeState.LSSpec

import edu.colorado.plv.bounder.lifestate.{FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSpace, ViewSpec}

In [3]:

// note: make sure to build the Infer Docker container: https://github.com/facebook/infer/tree/main/docker
val expDir = File(s"${System.getProperty("user.home")}/Desktop/")
val rc = RunConfig(apkPath="${baseDir}/target.apk", outFolder=Some("${baseDirOut}"))
def wcfg(name:String,apkPath:File) = name match{
    case x if x matches "AntennapodPlayerFragment_.*" => // _bug or _fix
        val tgt:InitialQuery = ReceiverNonNull(className="com.example.row1antennapodrxjava.ui.main.PlayerFragment", methodName="void call(java.lang.Object)",line=69,receiverMatcher=Some("act"))
//         val rc = RunConfig(apkPath="${baseDir}/target.apk", outFolder=Some("${baseDirOut}"), specSet=TestSpec("AntennaPod"), initialQuery= List(tgt))
        write(rc.copy(specSet=TestSpec("AntennaPod"), initialQuery=List(tgt)))
    case c if c matches "AntennapodExecute_bug" => ???
    case c if c matches "AntennapodExecute_fix" =>
        val tgt:InitialQuery = DisallowedCallin("com.example.row1antennapodrxjava.RemoverActivity$2", "void onClick(android.view.View)", SAsyncTask.disallowDoubleExecute)
        write(rc.copy(
                specSet=PickleSpec(Set(
                    ViewSpec.clickWhileNotDisabled,
                    LifecycleSpec.Activity_createdOnlyFirst), Set(SAsyncTask.disallowDoubleExecute)), 
                initialQuery=List(tgt)))
    case _ => throw new IllegalArgumentException(s"name: ${name} is not valid")
}

def setupTempDirForSimplified(name:String):File = {
    val workingDir = expDir / (name + "_workingdir")
//     if(workingDir.exists())
//         workingDir.delete()
    workingDir.createDirectoryIfNotExists()
    workingDir.glob("*").foreach{f => f.delete()}
    
    val baseSimpAppDir = File((System.getProperty("user.dir").split("/").dropRight(2) ++ Array("test_apps","simplified")).mkString("/"))
    val srcDir = baseSimpAppDir / name
    val wkSRC = workingDir / "src"
    srcDir.copyTo(wkSRC)
    val apkFile = srcDir.glob("**/*.apk").toList
    assert(apkFile.size == 1, "Please build sample app ${name} and ensure build generates apk file.")
    val wkAPK = workingDir / "target.apk"
    apkFile.head.copyTo(wkAPK)
    val wkCFG = workingDir / "cfg.json"
    wkCFG.overwrite(wcfg(name, wkAPK))
    workingDir
}
def runDocker(tmp:File, containerName:String, cmd:String) = {
    BounderUtil.runCmdFileOut(s"docker run --rm --mount type=bind,source=${tmp},target=/cwd -e BOUNDER_JAR='/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar' ${containerName} bash -c 'cd /cwd; ${cmd}'",tmp)
}
def runBounderOnTmp(tmp:File){
    runDocker(tmp, "bounder", "java -jar $BOUNDER_JAR -m verify -c cfg.json -b /cwd/ -u /cwd/")
    Driver.readDB(tmp)
    // TODO: interpret output
}

def runInferOnTmp(tmp:File){
    //TODO: call infer from docker
    //TODO: interpret infer results
    runDocker(tmp, "bounder", "cd src; infer run -- ./gradlew assembleDebug >infer.stdout 2>infer.stderr")
}
// TODO: interpret "proven up to k steps" for timeout
//TODO: generate latex table
//TODO: write full versions of other distilled apps

val expList = List("AntennapodPlayerFragment_bug", "AntennapodPlayerFragment_fix","AntennapodExecute_fix")

expDir: File = /Users/shawnmeier/Desktop
rc: RunConfig = RunConfig(
  "${baseDir}/target.apk",
  Some("${baseDirOut}"),
  None,
  TopSpecSet,
  List(),
  -1,
  5,
  "",
  600,
  true
)
defined function wcfg
defined function setupTempDirForSimplified
defined function runDocker
defined function runBounderOnTmp
defined function runInferOnTmp
expList: List[String] = List(
  "AntennapodPlayerFragment_bug",
  "AntennapodPlayerFragment_fix",
  "AntennapodExecute_fix"
)

In [ ]:
val f = setupTempDirForSimplified("AntennapodExecute_fix")
runBounderOnTmp(f)
//runInferOnTmp(f)


defined function tgt
res53_1: Any = "{\"apkPath\":\"/Users/shawnmeier/Documents/source/bounder/notebooks/distAppExp/dummyapk\",\"outFolder\":[\"/home\"],\"specSet\":\"testSpec:Antennapod\",\"initialQuery\":[{\"methodName\":\"void call(java.lang.Object)\",\"line\":69,\"t\":\"ReceiverNonNull\",\"matcher\":\"act\",\"className\":\"com.example.row1antennapodrxjava.ui.main.PlayerFragment\"}]}"

In [61]:
def matches(s:String) = s match {
    case x if x matches "foo|b" => true
    case _ => false
}

matches("foo")

defined function matches
res60_1: Boolean = true